# Prognostics for Turbofine Engines with 1D Convolutional Neural Networks 🩺

Prognostics is the prediction of Remaining useful life of instance of failure of a component based on the knowledge about current and future coditions of operation (obtained through various sensors or physical models).

The new C-MAPSS dataset DS02 from NASA provides degradation trajectories of 9 turbofan engines with unknown and different initial health condition for complete flights and two failure modes (HPT efficiency degradation & HPT efficiency degradation combined with LPT efficiency and capacity degradation). The data were synthetically generated with the Commercial Modular Aero-Propulsion System Simulation (C-MAPSS) dynamical model. The data contains multivariate sensors readings of the complete run-to-failure trajectories. Therefore, the records stop at the cycle/time the engine failed. A total number of 6.5M time stamps are available. Dataset copyright (c) by Manuel Arias.

**For training simplicity, the dataset has been preprocessed. The dataset has been downsampled from 1Hz to 0.1 Hz with an IIR 8th Order Chebyshev filter. Data format has been converted from double to float precison.**

## Imports 💼

In [ ]:
import os
import time 
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
from sklearn.preprocessing import MinMaxScaler
import torch.nn.functional as F

%matplotlib inline

## Download dataset 🔗

Download the dataset from [Google Drive Link](https://drive.google.com/file/d/1jXC3BQmEkupXkJbuXM52-Lf6ADLzR4Rr/view?usp=sharing) and put it in the folder of this notebook.

In [ ]:
folder = os.getcwd()
filename = f'{folder}/ncmapps_ds02.csv'
print(filename)

# Data exploration 🔍

<img src="images/cmapss.png" width="700"/>

### Feature descriptions

| Symbol |           Description           | Units |
|:------:|:-------------------------------:|:-----:|
|   Wf   |            Fuel flow            |  pps  |
|   Nf   |        Physical fan speed       |  rpm  |
|   Nc   |       Physical core speed       |  rpm  |
|   T24  | Total temperature at LPC outlet |   °R  |
|   T30  | Total temperature at HPC outlet |   °R  |
|   T48  | Total temperature at HPT outlet |   °R  |
|   T50  | Total temperature at LPT outlet |   °R  |
|   P15  |  Total pressure in bypass-duct  |  psia |
|   P2   |   Total pressure at fan inlet   |  psia |
|   P21  |   Total pressure at fan outlet  |  psia |
|   P24  |   Total pressure at LPC outlet  |  psia |
|  Ps30  |  Static pressure at HPC outlet  |  psia |
|   P40  | Total pressure at burner outlet |  psia |
|   P50  |   Total pressure at LPT outlet  |  psia |
|   alt  |             Altitude            |   ft  |
|  Mach  |        Flight Mach number       |   -   |
|   TRA  |     Throttle–resolver angle     |   %   |
|   T2   |  Total temperature at fan inlet |   °R  |
|  cycle |       Flight cycle number       |   -   |
|   Fc   |           Flight class          |   -   |
|   hs   |           Health state          |   -   |

In [ ]:
df = pd.read_csv(filename)
df.head()


In [ ]:
df.describe() 

In [ ]:
# Box Plots are usefull to see the data distribution and the outliers
df_columns_name = df.columns
numeric_columns = df.select_dtypes(include=['number']).columns.tolist()
print(numeric_columns)
#df.boxplot(column =numeric_columns, grid = False)

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
df.boxplot(numeric_columns[0] , ax=ax[0])
df.boxplot(numeric_columns[1:4], ax=ax[1])
df.boxplot(numeric_columns[4:8], ax=ax[2])
plt.suptitle('Test')
plt.show()

fig, ax = plt.subplots(1, 4, figsize=(10, 5))
df.boxplot(numeric_columns[8] , ax=ax[0])
df.boxplot(numeric_columns[9], ax=ax[1])
df.boxplot(numeric_columns[10], ax=ax[2])
df.boxplot(numeric_columns[11], ax=ax[3])
plt.suptitle('Test 2')
plt.show()

fig, ax = plt.subplots(1, 4, figsize=(10, 5))
df.boxplot(numeric_columns[12] , ax=ax[0])
df.boxplot(numeric_columns[13], ax=ax[1])
df.boxplot(numeric_columns[14], ax=ax[2])
df.boxplot(numeric_columns[15], ax=ax[3])
plt.suptitle('Test 3')
plt.show()

fig, ax = plt.subplots(1, 4, figsize=(10, 5))
df.boxplot(numeric_columns[16] , ax=ax[0])
df.boxplot(numeric_columns[17] , ax=ax[1])
df.boxplot(numeric_columns[18], ax=ax[2])
df.boxplot(numeric_columns[19], ax=ax[3])
plt.suptitle('Test 4')
plt.show()

fig, ax = plt.subplots(1, 3, figsize=(10, 5))
df.boxplot(numeric_columns[20], ax=ax[0])
df.boxplot(numeric_columns[21], ax=ax[1])
df.boxplot(numeric_columns[22], ax=ax[2])
plt.suptitle('Test 5')
plt.show()

### Flight Classes

The units are divided into three flight classes depending on whether the unit is operating short-length flights (i.e., flight class 1), medium-length flights (i.e., flight class 2), or long-length flights (i.e., flight class 2). A number of real flight conditions are available within each of the flight classes.

| Flight Class   | Flight Length [h]
| :-----------:  | :-----------:    
| 1              |    1 to 3        
| 2              |    3 to 5        
| 3              |    5 to 7        

In [ ]:
df.drop_duplicates(
    subset=['unit','Fc'], keep='last'
    ).plot(
        x='unit', y='Fc', 
        kind='bar', 
        xlabel='Unit # [-]', 
        ylabel='Flight Class # [-]'
    )

## Feature Overview

In [ ]:
LABELS = ['RUL']

Operative Conditions ($w$)

DASHlink- Flight Data For Tail 687.(2012). Retrieved on 2019-01-29 from https://c3.nasa.gov/dashlink/

In [ ]:
W_VAR = ['alt', 'Mach', 'TRA', 'T2']

Sensor readings ($X_s$)

In [ ]:
XS_VAR = ['T24', 'T30', 'T48', 'T50', 'P15', 'P2', 'P21', 'P24', 'Ps30', 'P40', 'P50', 'Nf', 'Nc', 'Wf']


In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 10))
sns.heatmap(df[W_VAR+XS_VAR+LABELS].corr(), cmap="vlag")
# Adding the LABELS to the heat map allows us to see which sensor is most correlated.
# The T48 and T50 sensors are the most correlated (monotonically) with RUL.

### Flight Traces
visualize a single flight trace of a given unit

In [ ]:
# We are trying to compare a flight during the first cycles measured and a flight during the last cycles with a similar altitude and speed curve for the same unit.
# Seein isn't easy to find
unit = 18
cycle = 3
df_u_sel = df.loc[(df.unit == unit) & (df.cycle == cycle)][W_VAR+XS_VAR+LABELS]# df.loc[(df.unit == unit) & (df.cycle == cycle)][W_VAR+XS_VAR]
df_u_sel.reset_index(inplace=True, drop=True)
axes = df_u_sel.plot(figsize=(12, 10), subplots=True, layout=(5, 4))

unit = 18
cycle = 70
df_u_sel = df.loc[(df.unit == unit) & (df.cycle == cycle)][W_VAR+XS_VAR+LABELS]# df.loc[(df.unit == unit) & (df.cycle == cycle)][W_VAR+XS_VAR]
df_u_sel.reset_index(inplace=True, drop=True)
axes = df_u_sel.plot(figsize=(12, 10), subplots=True, layout=(5, 4))

### Flight envelope

In [ ]:
f, ax = plt.subplots(1, 1)
for i in [3, 2, 1]:
    df.loc[df['Fc'] == i].plot(x='Mach', y='alt', alpha=0.8, label=f'Fc={i}', ax=ax, lw=5)
plt.xlabel('Mach Number - [-]')
plt.ylabel('Flight Altitude - [ft]')
plt.legend()

## Histogram of Flight Conditions

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(12, 10))
for i, var in enumerate(W_VAR):
    sns.kdeplot(data=df[W_VAR+['unit']], x=var, hue='unit',shade = True, gridsize=100, ax=axes[i//2][i%2], palette=sns.color_palette("husl", 9))

In [ ]:
# impact of cycle

filtered_unit = 5
filtered_df = df[df['unit'] == filtered_unit]

# Get the minimum and maximum values of 'cycle' within the first filter
min_cycle = filtered_df['cycle'].min()
max_cycle = filtered_df['cycle'].max()

plt.figure(figsize=(10, 6))

# Loop through each cycle in the filtered range and plot only every 10th cycle
for i in range(min_cycle, max_cycle + 1):
    if i % 10 == 0:  # Only plot every 10th cycle
        # Filter data for the current cycle
        cycle_data = filtered_df[filtered_df['cycle'] == i]
        
        # Define x-axis as the length of the cycle (0 to length of cycle_data - 1)
        x_values = range(len(cycle_data))
        y_values = cycle_data['T50']
        
        # Plot the curve for the current cycle
        plt.plot(x_values, y_values, label=f'Cycle {i}')

# Label the axes
plt.xlabel('number of measure')
plt.ylabel('T50')
plt.title(f'T50 evolution every 10th Cycle for the unit {filtered_unit}')
plt.legend()
plt.show()


# Developing your first prognostics model 💻
Step by step, you will learn how to build your first prognostics model from scratch!

## Define Sequence Dataset 


In [ ]:
class SlidingWindowDataset(Dataset):
    def __init__(self, dataframe, window=50, stride=1, horizon=1, device='cpu'):
        """Sliding window dataset with RUL label

        Args:
            dataframe (pd.DataFrame): dataframe containing scenario descriptors and sensor reading
            window (int, optional): sequence window length. Defaults to 50.
            stride (int, optional): data stride length. Defaults to 1.
            horizon (int, optional): prediction forcasting length. Defaults to 1.
        """
        self.window = window
        self.stride = stride
        self.horizon = horizon
        
        self.X = np.array(dataframe[XS_VAR+W_VAR].values).astype(np.float32)
        self.y = np.array(dataframe['RUL'].values).astype(np.float32)
        if 'ds' in dataframe.columns:
            unqiue_cycles = dataframe[['ds', 'unit', 'cycle']].value_counts(sort=False)
        else:
            unqiue_cycles = dataframe[['unit', 'cycle']].value_counts(sort=False)
        self.indices = torch.from_numpy(self._get_indices(unqiue_cycles)).to(device)

    # TODO add comment
    def _get_indices(self, unqiue_cycles):
        '''
        unqiue_cycles (pd.DataFrame): dataframe containing the values in columns 'ds'(if exist) , 'unit' , 'cycle' 
        and their occurance (the order of the datais preserved)

        This function return the indexes of each (start_recording_flight, end_recording_flight) pair in the overall dataframe
        Based on the 'unit' : motor ID and the 'cycle' : flight number
        Plus the 'ds' : ... (if exist)
        '''
        cycles = unqiue_cycles.to_numpy() 
        idx_list = []
        for i, c_count in enumerate(cycles):
            c_start = sum(cycles[:i]) 
            c_end = c_start + (c_count - self.window - self.horizon)
            if c_end + self.horizon < len(self.X): # handling y not in the last seq case
                idx_list += [_ for _ in np.arange(c_start, c_end + 1, self.stride)] 
        return np.asarray([(idx, idx+self.window) for idx in idx_list])

    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, i):
        i_start, i_stop = self.indices[i]
        x = self.X[i_start:i_stop, :]
        y = self.y[i_start]
        x = x.permute(1, 0)
        return x, y

In [ ]:
def create_datasets(df, window_size, train_units, test_units, device='cpu'):
    df_train = df[df['unit'].isin(train_units)]
    train_dataset = SlidingWindowDataset(df_train, window=window_size)

    df_test = df[df['unit'].isin(test_units)]    
    test_dataset = SlidingWindowDataset(df_test, window=window_size)

    # normalizing features
    scaler = MinMaxScaler()
    train_dataset.X = scaler.fit_transform(train_dataset.X)
    test_dataset.X = scaler.transform(test_dataset.X)

    # convert numpy array to tensors
    datasets = [train_dataset, test_dataset]
    for d in datasets:
        d.X = torch.from_numpy(d.X).to(device)
        d.y = torch.from_numpy(d.y).to(device)
    
    return datasets

def create_data_loaders(datasets, batch_size=256, val_split=0.2):
    # fixed seed for data splits for reproducibility
    random.seed(0)
    np.random.seed(0)
    
    d_train, d_test = datasets
    dataset_size = len(d_train)
    indices = list(range(dataset_size))
    split = int(np.floor(val_split * dataset_size))
    np.random.shuffle(indices)
    train_indices, val_indices = indices[split:], indices[:split]
    train_sampler = SubsetRandomSampler(train_indices)
    valid_sampler = SubsetRandomSampler(val_indices)

    train_loader = DataLoader(d_train, batch_size=batch_size, sampler=train_sampler)
    val_loader = DataLoader(d_train, batch_size=batch_size, sampler=valid_sampler)
    test_loader = DataLoader(d_test, batch_size=batch_size, shuffle=False)      

    d_info = f"train_size: {len(train_indices)}\t"
    d_info += f"validation_size: {len(val_indices)}\t"
    d_info += f"test_size: {len(d_test)}"
    print(d_info)
    return train_loader, val_loader, test_loader

## Define Trainer Class

In [ ]:
class Trainer:
    def __init__(
        self,
        model,
        optimizer,
        n_epochs=20,
        criterion=nn.MSELoss(),
        model_name='best_model',
        seed=42,
        device='cpu'
    ):
        self.seed = seed
        self.model = model
        self.optimizer = optimizer
        self.device = device
        self.n_epochs = n_epochs
        self.criterion = criterion
        
        # adding time_stamp to model name to make sure the save models don't overwrite each other, 
        # you can customize your own model name with hyperparameters so that you can reload the model more easily
        time_stamp = time.strftime("%m%d%H%M%S")#month,day,hour,minute,second
        self.model_path = f'models/{model_name}_{time_stamp}.pt'

        self.losses = {split: [] for split in ['train', 'eval', 'test']}
        
    def compute_loss(self, x, y, model=None):
        y = y.view(-1)
        y_pred = self.model(x)
        y_pred = y_pred.view(-1)
        loss = self.criterion(y, y_pred)
        return loss, y_pred, y
    
    def train_epoch(self, loader):
        self.model.train()
        # batch losses
        b_losses = []
        for x, y in loader:
            # Setting the optimizer gradient to Zero
            self.optimizer.zero_grad()
            x.to(torch.device(self.device))
            y.to(torch.device(self.device))
            
            loss, pred, target = self.compute_loss(x, y)
            
            # Backpropagate the training loss
            loss.backward()
            self.optimizer.step()
            b_losses.append(loss.detach().numpy())
        
        # aggregated losses across batches
        agg_loss = np.sqrt((np.asarray(b_losses) ** 2).mean())
        self.losses['train'].append(agg_loss)
        return agg_loss

    # decorator, equivalent to with torch.no_grad():
    @torch.no_grad()
    def eval_epoch(self, loader, split='eval'):
        self.model.eval()
        # batch losses
        b_losses = []
        for x, y in loader:
            x.to(torch.device(self.device))
            y.to(torch.device(self.device))
            
            loss, pred, target = self.compute_loss(x, y)
            
            b_losses.append(loss.detach().numpy())
        
        # aggregated losses across batches
        agg_loss = np.sqrt((np.asarray(b_losses) ** 2).mean())
        self.losses[split].append(agg_loss)
        return agg_loss
        
    def fit(self, loaders):
        print(f"Training model for {self.n_epochs} epochs...")
        train_loader, eval_loader, test_loader = loaders
        train_start = time.time()
        
        start_epoch = 0
        best_eval_loss = np.inf

        ##############
        patience = 10
        patience_counter = 0
        ##############
            
        for epoch in range(start_epoch, self.n_epochs):
            epoch_start = time.time()
            
            train_loss = self.train_epoch(train_loader)
            eval_loss = self.eval_epoch(eval_loader, split='eval')
            test_loss = self.eval_epoch(test_loader, split='test')

            if eval_loss < best_eval_loss:
                best_eval_loss = eval_loss
                self.save(self.model, self.model_path)
                ##############
                patience_counter = 0
            else:
                patience_counter += 1
                ##############

            s = (
                f"[Epoch {epoch + 1}] "
                f"train_loss = {train_loss:.5f}, "
                f"eval_loss = {eval_loss:.5f}, "
                f"test_loss = {test_loss:.5f}"
            )

            epoch_time = time.time() - epoch_start
            s += f" [{epoch_time:.1f}s]"
            print(s)

            ##############
            if patience_counter >= patience:
                print(f"Early stopping triggered.")
                train_time = int(time.time() - train_start)
                break
            ##############
    
        train_time = int(time.time() - train_start)
                
        print(f'Task done in {train_time}s')
    
    @ staticmethod
    def evaluate_model_performance(y_pred, y_true):
        """
        ## Task: Define Model Performance on RMSE and NASA score
        The performance of your implemented model should be evaluated using two common metrics applied in N-CMAPSS prognostics analysis:
        RMSE and NASA-score (in 1E5) as introduced in ["Fusing Physics-based and Deep Learning Models for Prognostics"](https://arxiv.org/abs/2003.00732)
        """
        # TODO: add implementation
        # Compute NASA score
        diff = y_pred - y_true
        alpha = np.where(diff > 0, 1/10, 1/13)

        # Calculate the NASA score based on the penalty factor and the magnitude of the error
        score = np.sum(np.exp(alpha * np.abs(diff)))

        # Normalize the NASA score as required
        score = score / 1e5

        # Compute RMSE (Root Mean Square Error)
        rmse = np.sqrt(np.mean((y_true - y_pred) ** 2))

        return score, rmse
    
    # decorator, equivalent to with torch.no_grad():
    @torch.no_grad()
    def eval_rul_prediction(self, test_loader):
        print(f"Evaluating test RUL...")
        
        ## MASK OUT EVAL and add explanation
        best_model = self.load(self.model) 
        best_model.eval()
        
        preds = []
        trues = []
        
        for x, y in tqdm(test_loader):
            x = x.to(self.device)
            y = y.to(self.device)

            _, y_pred, y_target = self.compute_loss(x, y)
            preds.append(y_pred.detach().cpu().numpy())
            trues.append(y_target.detach().cpu().numpy())
        
        preds = np.concatenate(preds, axis=0)
        trues = np.concatenate(trues, axis=0)
        
        df = pd.DataFrame({         
            'pred': preds,
            'true': trues,
            'err': np.sqrt((preds - trues)**2)
        })
        
        score, rmse = self.evaluate_model_performance(preds, trues)
        df_out = pd.DataFrame({
            'score': [score],
            'rmse': [rmse],
            'seed': [self.seed],
        })
        return df, df_out

    def save(self, model, model_path=None):
        os.makedirs(f'{folder}/models', exist_ok=True)
        if model_path is None:
            model_path = self.model_path 
        torch.save(model.state_dict(), model_path)
        
    def load(self, model, model_path=None):
        """
        loads the prediction model's parameters
        """
        if model_path is None:
            model_path = self.model_path
        model.load_state_dict(torch.load(model_path, map_location=self.device))
        print(f"Model {model.__class__.__name__} saved in {model_path} loaded to {self.device}")
        return model

    def plot_losses(self):
        """
        :param losses: dict with losses
        """
        linestyles = {
            'train': 'solid', 
            'eval': 'dashed', 
            'test': 'dotted', 
        }
        for split, loss in self.losses.items():
            ls = linestyles[split]
            plt.plot(range(1, 1+len(loss)), loss, label=f'{split} loss', linestyle=ls)
            plt.yscale('log')
                
        plt.title("Training/Validation Losses")
        plt.xlabel("Epoch")
        plt.ylabel("Loss")
        plt.legend()
        plt.show()


# Task: Implement a 1D Convolutional Neural Network (1DCNN) Model
Conventional CNN's developed for image tasks learn to extract features from the 2D input data. They are autonomous (require no domain expertise or prior info about the image) and thus can be applied to any image regardless of its dimensions. This is due to the fact that these CNN's go through an image by downsampling the image which we call straddling or windowing.  

Similarly 1D CNN learns to extract features from a time series data, by windowing over the data, considering a set of data observations each time. The benefit of using the CNN for sequence classification is that it can learn from the raw time series data, and in turn do not require domain expertise to engineer relevant features.

The CNN architecture outlined in the paper consists of five layers. The initial three layers are convolutional layers, each employing filters of size 10. The first two convolutional layers consist of ten channels each, while the third convolutional layer comprises a single channel. Zero padding is utilized to maintain the dimensions of the feature map throughout the network. Following the convolutional layers, the 2D feature map is flattened, leading into a 50-unit fully connected layer, and subsequently, a linear output neuron. The activation function used across the network is ReLU. The network encompasses 24,000 trainable parameters.

As an advanced extension of this task, you are encouraged to explore the inclusion of Dropout and Batch Normalization layers as regularization techniques to improve the model's generalization performance. Implementing these additional layers can help in reducing overfitting and ensuring that the model generalizes well to unseen data. Your exploration should evaluate the impact of these regularization techniques on the model's performance and compare the results with the baseline model (without Dropout and Batch Normalization).




<img src="images/assignment_1dcnn.png" width="700"/>

In [ ]:
def init_weights(m):
    if isinstance(m, nn.BatchNorm1d):
        m.weight.data.fill_(1.0)
        m.bias.data.zero_()
    elif isinstance(m, nn.Conv1d) or isinstance(m, nn.Linear):
        m.weight.data = nn.init.xavier_uniform_(
            m.weight.data, gain=nn.init.calculate_gain('relu'))
        if m.bias is not None:
            m.bias.data.zero_()


class CNN(nn.Module):
    '''
    A 1D-CNN model that is based on the paper "Fusing physics-based and deep learning models for prognostics"
    from Manuel Arias Chao et al. (with batchnorm layers)
    '''
    
    def __init__(self, 
                 in_channels=14, #corresponds to the number of sensor readings
                 out_channels=1, #RUL
                 window= 50 ,
                 n_ch=10, 
                 n_k=10, 
                 n_hidden=50, 
                 n_layers=3,
                 dropout=0.,
                 batch_norm = False,
                 padding='same'):
        """
        Args:
            n_features (int, optional): number of input features. Defaults to 18.
            window (int, optional): sequence length. Defaults to 50.
            n_ch (int, optional): number of channels (filter size). Defaults to 10.
            n_k (int, optional): kernel size. Defaults to 10.
            n_hidden (int, optional): number of hidden neurons for regressor. Defaults to 50.
            n_layers (int, optional): number of convolution layers. Defaults to 5.
        """
        super().__init__()
        # TODO: implement model architecture
        # Add convolution layers with BatchNorm and ReLU activations
        ######################
        # layers = []
        # for i in range(n_layers - 1):
        #     layers.append(nn.Conv1d(in_channels=in_channels if i == 0 else n_ch, 
        #                             out_channels=n_ch, 
        #                             kernel_size=n_k, 
        #                             padding=padding))
        #     layers.append(nn.ReLU())
        #     layers.append(nn.BatchNorm1d(n_ch))
        
        # # The third layer should have only one output channel (for the single-channel feature map)
        # layers.append(nn.Conv1d(in_channels=n_ch, 
        #                         out_channels=1, 
        #                         kernel_size=n_k, 
        #                         padding=padding))
        # layers.append(nn.ReLU())
        # layers.append(nn.BatchNorm1d(1))
        
        # # Convert the layers into a sequential block
        # self.conv_layers = nn.Sequential(*layers) # * to pass each layer separately 

        # # Define the fully connected layers
        # self.fc1 = nn.Linear(window, n_hidden)  # Input size is the flattened window size
        # self.fc2 = nn.Linear(n_hidden, out_channels)  # Output layer for regression (single output)

        # self.apply(init_weights)
        ######################
        ######################
        self.layer1 = nn.Sequential(
            nn.Conv1d(in_channels=in_channels, 
                      out_channels=n_ch,
                      kernel_size=n_k,
                      padding=padding),
            nn.ReLU(),
            nn.BatchNorm1d(n_ch) if batch_norm else nn.Identity())
        self.layer2 = nn.Sequential(
            nn.Conv1d(in_channels=n_ch, 
                      out_channels=n_ch,
                      kernel_size=n_k,
                      padding=padding),
            nn.ReLU(),
            nn.BatchNorm1d(n_ch) if batch_norm else nn.Identity()) 
        self.layer3 = nn.Sequential(
            nn.Conv1d(in_channels=n_ch, 
                      out_channels=out_channels,
                      kernel_size=n_k,
                      padding=padding),
            nn.ReLU(),
            nn.BatchNorm1d(1) if batch_norm else nn.Identity())
        self.fc1 = nn.Sequential(
            nn.Linear(window, n_hidden),
            nn.ReLU(),
            nn.Dropout(dropout))
        self.fc2 = nn.Sequential(
            nn.Linear(n_hidden,out_channels))
        self.apply(init_weights)
        ##########################

        
    def forward(self, x):
        """
        Forward pass through the network.

        Args:
            x (torch.Tensor): Input data of shape (batch_size, in_channels, sequence_length)

        Returns:
            torch.Tensor: Output prediction for RUL (Remaining Useful Life)
        """
        # TODO: implement forward pass
        ######################
        # # 3 convolutional layers 
        # x = self.conv_layers(x)

        # # Flatten the output of the last convolutional layer
        # x = x.view(x.size(0), -1)

        # #################
        # # for me the flatten part should be between the last two layers and not before
        # # and I am not convince by the two last line 
        # #################

        # # Fully connected layers
        # x = F.relu(self.fc1(x)) # straightforward activation function to tensors
        # x = self.fc2(x)
        ######################
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = x.flatten(start_dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        

        return x
        

# Training a model instance

In [ ]:
def seed_everything(seed: int):
    r"""Sets the seed for generating random numbers in PyTorch, numpy and
    Python.

    Args:
        seed (int): The desired seed.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [ ]:
# dataset parameters
TRAIN_UNITS = [2, 5, 10, 16, 18, 20]
TEST_UNITS = [11, 14, 15]

DEFAULT_PARAMS = {
    # CNN model parameters
    'in_channels': 18, 
    'out_channels': 1,
    'window': 50,
    'n_ch': 10, 
    'n_k': 10, 
    'n_hidden': 50, 
    'n_layers': 3,
    'batch_norm': False,
    'dropout': 0., 
    'padding': 'same', 
    # training parameters
    'batch_size': 256,
    'base_lr': 1e-3,
    'weight_decay': 1e-5,
    'max_epochs': 1 #50
}

DATASETS = create_datasets(df, window_size=DEFAULT_PARAMS['window'], train_units=TRAIN_UNITS, test_units=TEST_UNITS)
LOADERS = create_data_loaders(DATASETS, batch_size=DEFAULT_PARAMS['batch_size'], val_split=0.2)


In [ ]:
SEED = 42
seed_everything(SEED)

model = CNN(
  in_channels=DEFAULT_PARAMS['in_channels'],
  out_channels=DEFAULT_PARAMS['out_channels'],
  n_ch=DEFAULT_PARAMS['n_ch'],
  n_k=DEFAULT_PARAMS['n_k'],
  n_hidden=DEFAULT_PARAMS['n_hidden'],
  n_layers=DEFAULT_PARAMS['n_layers'],
  batch_norm=DEFAULT_PARAMS['batch_norm']
  dropout=DEFAULT_PARAMS['dropout'],
)
print(model)

optimizer = torch.optim.Adam(
  model.parameters(),
  lr=DEFAULT_PARAMS['base_lr'],
  weight_decay=DEFAULT_PARAMS['weight_decay'],
)

criterion = nn.MSELoss()
trainer = Trainer(
  model,
  optimizer,
  criterion=criterion,
  n_epochs=DEFAULT_PARAMS['max_epochs'],
  seed=SEED,
)

trainer.fit(LOADERS)

In [ ]:
trainer.plot_losses()

## Evaluation on the test set

In [ ]:
df_test, df_out = trainer.eval_rul_prediction(LOADERS[-1])

In [ ]:
df_test.head()

In [ ]:
df_test.plot(y=['true', 'pred'])

## Effect of Dropout and Batch normalization

In [ ]:
# IDEA GENERAL

params = {
    # CNN model parameters
    'in_channels': 18, 
    'out_channels': 1,
    'window': 50,
    'n_ch': 10, 
    'n_k': 10, 
    'n_hidden': 50, 
    'n_layers': 3,
    'batch_norm':False, # to be changed
    'dropout': 0., # to be changed
    'padding': 'same', 
    # training parameters
    'batch_size': 256,
    'base_lr': 1e-3,
    'weight_decay': 1e-5,
    'max_epochs': 3 
}



# Experiment loop, we remove the default setting 
configurations = [
    {"batch_norm": True, "dropout": 0.0},      # Batch norm only
    {"batch_norm": False, "dropout": 0.1},     # Dropout only (e.g., 0.3)
    {"batch_norm": True, "dropout": 0.1},       # Both dropout and batch norm
    {"batch_norm": False, "dropout": 0.2},     
    {"batch_norm": True, "dropout": 0.2},   
    {"batch_norm": False, "dropout": 0.3},     
    {"batch_norm": True, "dropout": 0.3},
    {"batch_norm": False, "dropout": 0.4},     
    {"batch_norm": True, "dropout": 0.4},  
    {"batch_norm": False, "dropout": 0.5},     
    {"batch_norm": True, "dropout": 0.5}  
]

results = []
for config in configurations:
    print(f"Running configuration: BatchNorm={config['batch_norm']}, Dropout={config['dropout']}")
    seed_everything(SEED)
    model = CNN(
        in_channels=params['in_channels'],
        out_channels=params['out_channels'],
        n_ch=params['n_ch'],
        n_k=params['n_k'],
        n_hidden=params['n_hidden'],
        n_layers=params['n_layers'],
        batch_norm=config['batch_norm'],
        dropout=config["dropout"], # here I change the drop out to the one in config
    )

    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=params['base_lr'],
        weight_decay=params['weight_decay'],
    )
    criterion = nn.MSELoss()
    trainer = Trainer(
        model,
        optimizer,
        criterion=criterion,
        n_epochs=params['max_epochs'],
        seed=SEED,
    )
    trainer.fit(LOADERS)

    df_eval, df_eval_out = trainer.eval_rul_prediction(LOADERS[1])
    df_test, df_test_out = trainer.eval_rul_prediction(LOADERS[2])

    # extract the metrics of the eval 
    results.append({
        "config": config,
        "score": df_eval_out['score'].values[0],
        "rmse": df_eval_out['rmse'].values[0]
    })



# Sort the configurations by RMSE (ascending order for better performance)
sorted_results = sorted(results, key=lambda x: x['rmse'])

# Get the top 3 configurations based on RMSE
top_three_configs = sorted_results[:3]


params_combination_1 = {
    # CNN model parameters
    'in_channels': 18, 
    'out_channels': 1,
    'window': 50,
    'n_ch': 10, 
    'n_k': 10, 
    'n_hidden': 50, 
    'n_layers': 3,
    'batch_norm': top_three_configs[0]['config']['batch_norm'], 
    'dropout': top_three_configs[0]['config']['dropout'], 
    'padding': 'same', 
    # training parameters
    'batch_size': 256,
    'base_lr': 1e-3,
    'weight_decay': 1e-5,
    'max_epochs': 50  # You can adjust this as needed
}

params_combination_2 = {
    # CNN model parameters
    'in_channels': 18, 
    'out_channels': 1,
    'window': 50,
    'n_ch': 10, 
    'n_k': 10, 
    'n_hidden': 50, 
    'n_layers': 3,
    'batch_norm': top_three_configs[1]['config']['batch_norm'], 
    'dropout': top_three_configs[1]['config']['dropout'], 
    'padding': 'same', 
    # training parameters
    'batch_size': 256,
    'base_lr': 1e-3,
    'weight_decay': 1e-5,
    'max_epochs': 50  # You can adjust this as needed
}

params_combination_3 = {
    # CNN model parameters
    'in_channels': 18, 
    'out_channels': 1,
    'window': 50,
    'n_ch': 10, 
    'n_k': 10, 
    'n_hidden': 50, 
    'n_layers': 3,
    'batch_norm': top_three_configs[2]['config']['batch_norm'], 
    'dropout': top_three_configs[2]['config']['dropout'], 
    'padding': 'same', 
    # training parameters
    'batch_size': 256,
    'base_lr': 1e-3,
    'weight_decay': 1e-5,
    'max_epochs': 50  # You can adjust this as needed
}

params_combinations = [params_combination_1, params_combination_2, params_combination_3]

results_long_run = []
for parameters_ in params_combinations:
    seed_everything(SEED)
    model = CNN(
        in_channels=parameters_['in_channels'],
        out_channels=parameters_['out_channels'],
        n_ch=parameters_['n_ch'],
        n_k=parameters_['n_k'],
        n_hidden=parameters_['n_hidden'],
        n_layers=parameters_['n_layers'],
        batch_norm=parameters_['batch_norm'],
        dropout=parameters_["dropout"], 
    )
    
    optimizer = torch.optim.Adam(
        model.parameters(),
        lr=parameters_['base_lr'],
        weight_decay=parameters_['weight_decay'],
    )
    criterion = nn.MSELoss()
    trainer = Trainer(
        model,
        optimizer,
        criterion=criterion,
        n_epochs=parameters_['max_epochs'],
        seed=SEED,
    )
    trainer.fit(LOADERS)

    df_eval, df_eval_out = trainer.eval_rul_prediction(LOADERS[1])
    df_test, df_test_out = trainer.eval_rul_prediction(LOADERS[2])

    # extract the metrics of the eval 
    results_long_run.append({
        "config": config,
        "score": df_eval_out['score'].values[0],
        "rmse": df_eval_out['rmse'].values[0]
    })
print(results_long_run)
indexed_scores = list(enumerate(results_long_run))
sorted_scores = sorted(indexed_scores, key=lambda x: x[1]['rmse'])
sorted_results_and_index = sorted(results_long_run, key=lambda x: x['rmse'])
top_indices = [index for index, score in sorted_scores[:3]]
best_index = top_indices[0]
BEST_PARAMS = params_combinations[best_index]
print(BEST_PARAMS)




## Aggregate results of multiple runs

In [ ]:
# Update this model with the most relevent previous parameter
def run_single(seed, params=BEST_PARAMS):
  seed_everything(seed)

  model = CNN(
    in_channels=params['in_channels'],
    out_channels=params['out_channels'],
    n_ch=params['n_ch'],
    n_k=params['n_k'],
    n_hidden=params['n_hidden'],
    n_layers=params['n_layers'],
    batch_norm=params['batch_norm'],
    dropout=params['dropout'],
  )

  optimizer = torch.optim.Adam(
    model.parameters(),
    lr=params['base_lr'],
    weight_decay=params['weight_decay'],
  )

  criterion = nn.MSELoss()
  trainer = Trainer(
    model,
    optimizer,
    criterion=criterion,
    n_epochs=params['max_epochs'],
    seed=seed,
  )

  trainer.fit(LOADERS)
  df_eval, df_eval_out = trainer.eval_rul_prediction(LOADERS[1])
  df_test, df_test_out = trainer.eval_rul_prediction(LOADERS[2])
  return df_eval, df_eval_out, df_test, df_test_out

N_RUNS = 5
eval_score = []
eval_rmse = []
test_score = []
test_rmse = []
predictions_df = pd.DataFrame()  # DataFrame to store errors with columns for each run

for i in range(SEED, SEED+N_RUNS):
  df_eval, df_eval_out, df_test, df_test_out = run_single(i)

  eval_score.append(df_eval_out['score'])
  eval_rmse.append(df_eval_out['rmse'])
  test_score.append(df_test_out['score'])
  test_rmse.append(df_test_out['rmse'])

  # Calculate the prediction for df_test and accumulate them
  predictions = df_test['pred']-df_test['true']  # Calculate prediction
  predictions_df[f'#run{i+1}'] = predictions.values   # Append predictions to cumulative array



# Compute mean and standard deviation for RMSE and score

# Function to calculate mean and standard deviation
def calc_stats(data):
    mean = np.mean(data)
    std_dev = np.std(data)
    return mean, std_dev

# Calculate for each list
eval_score_mean, eval_score_std = calc_stats(eval_score)
eval_rmse_mean, eval_rmse_std = calc_stats(eval_rmse)
test_score_mean, test_score_std = calc_stats(test_score)
test_rmse_mean, test_rmse_std = calc_stats(test_rmse)

# Print results
print(f"Evaluation Score - Mean: {eval_score_mean}, Std Dev: {eval_score_std}")
print(f"Evaluation RMSE - Mean: {eval_rmse_mean}, Std Dev: {eval_rmse_std}")
print(f"Test Score - Mean: {test_score_mean}, Std Dev: {test_score_std}")
print(f"Test RMSE - Mean: {test_rmse_mean}, Std Dev: {test_rmse_std}")



## Evaluation of the performence per unit

In [ ]:

# Step 1: Identify change indices in `df_test['true']`
change_indices = df_test.index[(df_test['true'].shift(1) == 0) & (df_test['true'] > 0)].tolist()

# Add the first and last index to cover all intervals
split_indices = np.concatenate(([0], change_indices, [len(df_test)]))

# Cycle indices
cycle_indices = np.where(df_test['true'].diff() != 0)[0]
cycle_indices = np.concatenate(( cycle_indices, [len(df_test)]))

# Step 2: Separate `predictions_df` into a dictionary based on unique intervals
predictions_dict_unit = {i: predictions_df.iloc[split_indices[i]:split_indices[i+1]] for i in range(len(split_indices) - 1)}

# Initialize the plot
plt.figure(figsize=(10, 6))

# Step 3: Calculate and plot mean, standard deviation, and confidence intervals for each unit segment
colors = ['#1f77b4', '#ff7f0e', '#2ca02c']  # Define colors for each unit

for index, predictions_df_unit in predictions_dict_unit.items():
    # Calculate mean and confidence interval for each point in the segment
    mean_prediction = predictions_df_unit.mean(axis=1)  # Mean prediction at each point
    std_prediction = predictions_df_unit.std(axis=1)    # Standard deviation at each point
    
    confidence_interval = 1.96 * std_prediction / np.sqrt(N_RUNS)  # 95% confidence interval
    
    # filter the cycle_indices
    predictions_range_start = predictions_df_unit.index[0]
    predictions_range_end = predictions_df_unit.index[-1]
    cycle_indices_in_range = [index for index in cycle_indices if predictions_range_start <= index <= predictions_range_end+1]
    cycle_indices_in_range[-1] -= 1 # manual correction
    # Assuming cycle_indices_in_range and predictions_df_unit are already defined
    mean_predictions = []
    min_interval_predictions = []
    max_interval_predictions = []
    # Loop through cycle indices to compute means for each segment
    for i in range(len(cycle_indices_in_range) - 1):
        start_index = cycle_indices_in_range[i]
        end_index = cycle_indices_in_range[i + 1]
        
        # Slice predictions_df_unit based on the indices
        # Ensure that you slice only within the bounds of the DataFrame
        segment = predictions_df_unit.loc[start_index:end_index - 1]  # End index is exclusive
        segment_mean = mean_prediction.loc[start_index:end_index - 1]
        segment_confidence_interval = confidence_interval.loc[start_index:end_index - 1]
        
        
        # Compute the mean for the current segment
        mean_segment = segment_mean.mean() # Get the mean value (assuming 1 column)
        mean_predictions.append(mean_segment)
        std_prediction = segment.std().values[0] # Standard deviation at each point
        min_interval_segment =  min(segment_mean - segment_confidence_interval)
        
        max_interval_segment =  max(segment_mean + segment_confidence_interval)
        min_interval_predictions.append(min_interval_segment)
        max_interval_predictions.append(max_interval_segment)



   
    # Define x values starting from 0 for each segment
    x_values = np.arange(len(mean_predictions))

    # Plot mean prediction with confidence interval for the current segment
    unit_label = f"Unit {TEST_UNITS[index]}"  # Label for the current unit
    plt.plot(x_values, mean_predictions, label=f'{unit_label}', color=colors[index], alpha=0.9, linewidth=1.5, marker='o', markersize=4)  
    plt.fill_between(
        x_values,
        min_interval_predictions,
        max_interval_predictions,
        color=colors[index],
        alpha=0.2  # Keep the confidence interval fill transparency
    )    

# Add ±5 horizontal lines with a dashed red line
plt.axhline(y=5, color='red', linestyle='-.', linewidth=1, label=r'$\epsilon = \pm 5$')
plt.axhline(y=-5, color='red', linestyle='-.', linewidth=1)

# Add a solid line at 0 for reference
plt.axhline(y=0, color='red', linestyle='-', linewidth=2)

# Formatting the plot
plt.xlabel('Time [cycles]')
plt.ylabel('Error RUL [cycles]')
plt.legend()

plt.show()

## Bonus: Hyperparameter tuning

**Model parameters:** These are the parameters that are estimated by the model from the given data. For example the weights of a deep neural network. 

**Model hyperparameters:** These are the parameters that cannot be estimated by the model from the given data. These parameters are used to estimate the model parameters. For example, the learning rate in deep neural networks.

*Hyperparameter tuning* (or hyperparameter optimization) is the process of determining the right combination of hyperparameters that maximizes the model performance. It works by running multiple trials in a single training process. Each trial is a complete execution of your training application with values for your chosen hyperparameters, set within the limits you specify. This process once finished will give you the set of hyperparameter values that are best suited for the model to give optimal results.

There are many python libraries for hyperparameter tuning:
1. RayTune
2. Optuna
3. Hyperopt
4. sklearn
etc.

Here we will build a simple tuner using Optuna

In [ ]:
!pip install -U optuna

There are several ways of hyperparameter tuning (all the above mentioned libraries apply several or all of those):

GridSearch : We create a discrete search space of hyperparameters to be tuned. This descrete search space grid is exhaustively searched for the best combination.

RandomSearch : We define distributions for each hyperparameter. Here the key difference is not all values are tested and values are selected at random. Since randomsearch does not test all hyperparameter values it does not necessarily return the best performing parameters, but it returns a good performing model in significantly shorter time

In [ ]:
import optuna

This is an example of random search, Optuna Tune uses Bayesian optimization by default, however for this time we use the random search algorithm.


In [ ]:
def evaluate_model(params):
    n_runs = 3 # number of runs to average over, you can decrease this number to speed up the optimization
    df_all_val = pd.DataFrame()
    for i in n_runs:
        seed = SEED + i
        df_eval, df_eval_out, df_test, df_test_out = run_single(seed, params)
        df_all_val = df_all_val.append(df_eval_out) # append the validation results
    # average over n_runs of the validation results, we use the mean of the rmse as the objective to minimize
    # we use the validation results for hyperparameter tuning
    rmse = df_all_val['rmse'].mean()
    return rmse

In [ ]:
def objective(trial):
    # We can add all the parameters that use for defining the model and trainer or even the dataset builder.

    # we define the parameter space, you can add more parameters to tune or reduce, you can also start with the best hyperparameters defined in the paper and finetune
    params = DEFAULT_PARAMS.copy()
    params['base_lr'] = trial.suggest_categorical('base_lr', [0.001, 0.01, 0.1])
    params['n_layer'] = trial.suggest_categorical('kernel_size ', [3, 4, 5])
    params['kernel_size'] = trial.suggest_categorical('kernel_size ', [3, 5, 7])
    ...
    
    out = evaluate_model(params)
    # We need to minimize the predicted rmse.
    rmse = out['rmse']
    return rmse 


In [ ]:
# define a study name, otherwise a random name will be generated
study_name = 'hypertune_1'

In [ ]:
n_trials = 2 # number of set of hyperparameters to train
# based on the output of the objective, we either maximize or minimize. If we returned accuracy, we would be maximize.    
study = optuna.create_study(f'sqlite:///{folder}/study.db',
                            study_name=study_name,
                            direction="minimize",
                            load_if_exists=True,)
study.optimize(objective, n_trials=n_trials)

In [ ]:
# Load the study for resuming, comment out when reloading
# study = optuna.load_study(study_name=study_name, storage=f'sqlite:///{folder}/study.db')

### Visualizing impact of hyperparameters

In [ ]:
# You can visualize the importance of hyperparameters
fig = optuna.visualization.plot_param_importances(study)
fig.show()

In [ ]:
# Contour plot between two hyperparameters
fig = optuna.visualization.plot_contour(study, params=['lr', 'nl'])
fig.show()

In [ ]:
# you can get a dataframe of the hyperparamter results with:
hyper_df = study.trials_dataframe()
hyper_df

### Get the best model parameter

In [ ]:
# Get the est model parameter
best_trial = study.best_trial
for key, value in best_trial.params.items():
    print(f"{key}: {value:.5f}")

# Assignment 3: Prognostics for Turbofine Engines with 1D Convolutional Neural Networks 🩺

## I. Task Description:

### Background:
Conventional CNNs, primarily developed for image analysis, have shown immense capability in autonomously extracting features from 2D data. This autonomy, achieved through striding or windowing, enables the application of CNNs across various image dimensions without requiring domain expertise. 

Similarly, 1D CNNs have demonstrated effectiveness in time series data analysis by employing a windowing technique to scan through sequential data, extracting valuable features autonomously. This characteristic of 1D CNNs makes them a powerful tool for prognostics, a field concerned with predicting the Remaining Useful Life (RUL) of systems based on historical and real-time operational data.

### Problem Formulation 🚀:
In this task, the objective is to delve into the domain of prognostics to address a real-world problem. Utilizing the architecture described in the paper: "Fusing Physics-based and Deep Learning Models for Prognostics" ([link](https://arxiv.org/abs/2003.00732)), you have to develop a 1D CNN model to predict the Remaining Useful Life (RUL) of certain units. The provided dataset comprises time series data from different units, and your goal is to accurately estimate the RUL for specified test units.

Training units (engines) **2, 5, 10, 16, 18, 20** 

Test units (engines) **11, 14, 15** 

### Exploratory Data Analysis:
1. Explore and understand the provided dataset, identifying the key features that could be indicative of a unit's health and consequently its RUL **(10 points max)**.


### Implementation:
1. Implement the 1D CNN model as per the referenced paper **(20 points max)**. Be careful with the tensor dimensions that you pass to 1DCNN. It should have (n samples, n channels, n timesteps) dimensions.
2. Investigate the effect of Dropout and Batch Normalization layers on the model's performance as regularization techniques to mitigate overfitting **(10 points max)**.

### Interpretation:
1. How do the predictions of RUL evolve over time, within a cycle (i.e a complete flight cycle consisting of taking off, crusing and landing) and between cycles, for a given unit? Visualize and discuss **(30 points max)**.
2. How well does your model generalizes to the test units? Identify possible reasons why the model may perform better on some units and worse on others **(20 points max)**. 

### Evaluation:
The performance of your implemented model should be rigorously evaluated across 5 independent runs with different random seeds to ensure the robustness of the results. 
For each run, utilize the following two common metrics applied in C-MAPSS prognostics analysis:

The performance of your implemented model should be evaluated using two common metrics applied in C-MAPSS prognostics analysis:
1. **Root Mean Square Error (RMSE)**:

$\text{RMSE} = \sqrt{\frac{1}{m^*} \sum_{j=1}^{m^*} (\Delta(j))^2}$

where $m^*$ denotes the total number of test data samples, and $\Delta(j)$ is the difference between the estimated and the real Remaining Useful Life (RUL) of the $j$-th sample, i.e., $y(j) - \hat{y}(j)$.

2. **NASA's Scoring Function (s)**:
$s = \sum_{j=1}^{m^*} \exp\left(\alpha \cdot |\Delta(j)|\right) $
where $\alpha = \frac{1}{13}$ if RUL is underestimated, and $\alpha = \frac{1}{10}$ otherwise. The scoring function $s$ is not symmetric and penalizes over-estimation more than under-estimation.


Aggregate the results from all **5 runs** to provide a mean and standard deviation for both the RMSE and NASA's Scoring Function. This aggregated evaluation will provide a more robust understanding of the model's performance on the selected prognostics task, comparing it to purely data-driven deep learning models as outlined in the referenced paper.

We provide in total **(10 points max)** for proper evaluation of the model. 

### Bonus:
1. Apply hyperparameter tuning with grid search to improve the model performance. You can start with the best hyperparameters from the paper **(10 points max)**. 
2. Explore alternative architectures or approaches to enhance the prognostic performance further **(10 points max)**.

N.B. The maximum of points can not exceed 100 points together with the bonus task.

## II. Report Submission:

Upon completion of the implementation and evaluation tasks, you are required to submit a comprehensive report summarizing your work and findings. Your report should span 5-8 pages and should be structured as follows:

#### Introduction:
- A brief introduction/literature review of 1D Convolutional Neural Networks (1DCNN) and their application in time series data analysis, particularly in prognostics.

#### Model Description:
- A detailed description of the 1DCNN model you implemented, including the architecture, layers, and any regularization techniques used.
- Mention of the hyperparameters that were tuned, and any additional modifications made to the original model architecture from the paper.

#### Methodology:
- Description of the data preparation process, training, and evaluation methodologies.
- Mention the training and testing units used as specified in the task.

#### Results and Discussion:
- Summarization of the evaluation results across 5 runs, providing the mean and standard deviation for both RMSE and NASA's Scoring Function.
- Evaluation of the performance per unit, and comparison of your model's performance with the results reported in the referenced paper.
- Discussion on the impact of Dropout and Batch Normalization (if applied), and any other observations regarding the model's performance.
- Any challenges faced during the implementation and how they were addressed.

#### Conclusion:
- Summary of key findings, lessons learned, and suggestions for future work to possibly improve the model's performance.

#### References:
- Proper citation of the paper, any other related works, and resources utilized in completing the assignment.

Ensure your report is well-organized, clear, and concise. Visual aids like graphs, tables, and diagrams should be used to enhance the explanation of your work, and comparisons made. The discussion should provide insightful analysis on the model's performance and a critical comparison with the paper's results. The report should be submitted in PDF format by **6.11.2024 Midnight 23:59**.
